# Nonlinear hardening - rate effect

In [ ]:
%matplotlib widget
from bmcs_matmod.api import GSMRM
import matplotlib.pylab as plt 
import sympy as sp
import numpy as np
sp.init_printing()

In [ ]:
gsm_F = GSMRM.load_from_disk('gsm_F_1d_t_e_vp_d')

In [ ]:
gsm_F.F_expr

In [ ]:
gsm_F.f_expr

In [ ]:
gsm_F.phi_ext_expr

## Monotonic loading

In [ ]:
_f_c = 44
_f_t = -0.1 * _f_c
_X_0 = (_f_c + _f_t) / 2
_f_s = (_f_c - _f_t) / 2
_E = 50000
_KH_factor = 1
_KH = _E * _KH_factor
_K_ratio = 0.7 # 0.01 # 0.015
_K = _KH * _K_ratio
_H = _KH * (1 - _K_ratio)
material_params = dict(
    E_=_E, 
    gamma_lin_= _H, # _E * 10, 
    gamma_exp_= 0.5,
    alpha_0_ = 0.003,
    K_lin_= _K, # _E / 5,
    k_exp_= 0.5,
    z_0_ = 0.003,
    S_=0.8,
    c_=2.5 * 3,
    r_=2.7,
    f_c_=_f_s,
    X_0_=_X_0,  
    eta_=20000,
    T_0_=20,
    C_v_=0.01, # 0.0001, 
    beta_=0.0001,
    alpha_therm_=0, # 1.2e-5,
    d_N_ = 1
)
material_params

In [ ]:
def gsm_run(gsm_, u_ta, T_t, t_t, **material_params):
    response = gsm_.get_response(u_ta, T_t, t_t, **material_params)
    _t_t, _u_tIa, _T_t, _Eps_tIb, _Sig_tIb, _iter_t, _dF_dEps_t, lam_t = response
    _u_atI, _Eps_btI, _Sig_btI, _dF_dEps_btI = [np.moveaxis(v_, -1, 0) for v_ in (_u_tIa, _Eps_tIb, _Sig_tIb, _dF_dEps_t)]
    _sig_atI = gsm_.get_sig(_u_atI, _T_t, _Eps_btI, _Sig_btI, **material_params )
    return _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI, lam_t 

gsm_F.vp_on = True
gsm_F.update_at_k = False

def get_cyclic_load(max_s, max_t, n_cycles, n_incr):
    # Generating loading history
    s_arr = np.tile(np.array([-1, 1]), n_cycles) * np.linspace(0, max_s, 2 * n_cycles)
    s_arr = np.interp(np.linspace(0, max_t, n_incr * len(s_arr)), np.linspace(0, max_t, len(s_arr)), s_arr)

    # time array as input
    t_arr = np.linspace(0, max_t, len(s_arr))
    return s_arr, t_arr

In [ ]:
response_values = {}
max_s = 0.01
dot_s_list = [0.00005, 0.0005, 0.005]
for dot_s in dot_s_list:
#for dot_s in [0.01, 0.02, 0.05]: 
    print('dot_s', dot_s)
    u_t, t_t = get_cyclic_load( n_cycles=1, max_s=max_s, 
                                  max_t=max_s/dot_s, n_incr = 200)
    response_values[dot_s] = gsm_run(gsm_F, u_t[:,np.newaxis], 20 + 0 * t_t, t_t, **material_params)

In [ ]:
from scipy.integrate import cumtrapz
fig, ((ax, ax_T, ax_Diss), (ax_u_p, ax_lam, ax_omega)) = plt.subplots(2,3, figsize=(12,6), tight_layout=True)
ax_z = ax_lam.twinx()

for dot_s, resp in response_values.items():
    
    label = f'$\dot{{s}}$ = {dot_s}'
    _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI, lam_t = resp 
    _u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_F.Eps_as_blocks(_Eps_btI)
    _, _Z_atI, _X_atI, _Y_atI = gsm_F.Eps_as_blocks(_Sig_btI)
    ax.plot(_u_atI[0, :, 0], _sig_atI[0, :, 0], label=label);
    ax.legend()
    ax.set_title(r'stress-strain')
    ax.set_ylabel(r'$\varsigma$')
    ax.set_xlabel(r'$\varepsilon$')

    ax_T.plot(_u_atI[0, :, 0], _T_t, label=label);
    ax_T.legend()
    ax_T.set_title(r'temperature')
    ax_T.set_ylabel(r'$\vartheta$')
    ax_T.set_xlabel(r'$\varepsilon$')

    Diss_btI_F = cumtrapz(_dF_dEps_btI, _Eps_btI, initial=0, axis=1)
    ax_Diss.plot(_t_t, np.sum(Diss_btI_F[...,0], axis=0), alpha=1, label='F')
    ax_Diss.set_title(r'dissipation')

    r = material_params['r_']
    c = material_params['c_']
    ax_u_p.plot(_u_atI[0, :, 0], (1 - _omega_atI[0, :, 0]**c)* (_Y_atI[0, :, 0] / material_params['S_'])**r)
    ax_u_p.set_xlabel(r'$\varepsilon$/-')
    ax_u_p.set_ylabel(r'$\varepsilon_\mathrm{p}$/-')
    ax_u_p.set_title(r'$\partial{\varphi}/{\partial Y} =  (1-\omega)^c (Y/S)^r$')

    ax_lam.plot(_u_atI[0, :, 0], lam_t)
    ax_lam.set_xlabel(r'$\varepsilon$/-')
    ax_lam.set_ylabel(r'$\lambda$/-')
    ax_lam.set_title(r'plastic multiplier: $\dot{\lambda}$')

    ax_z.plot(_u_atI[0, :, 0], _Z_atI[0, :, 0], ls='dashed')
    ax_z.set_ylabel(r'$z$/-')

    ax_omega.plot(_u_atI[0, :, 0], _omega_atI[0, :, 0])
    ax_omega.set_xlabel(r'$\varepsilon$/-')
    ax_omega.set_ylabel(r'$\omega$/-')
    ax_omega.set_title(r'damage: $\omega =  \dot{\lambda} \cdot \partial\varphi / \partial Y$')